In [72]:
import pandas as pd
import altair as alt 
from vega_datasets import data

## Aplicando connhecimento e diferentes funcionalidades

Nessa seção, nosso objetivo foi apresentar exemplos aplicando os aprendizados do capítulo juntamente com outras ferramentas das bibliotecas Altair e Pandas. Tais gráficos visam a ampliar o repertório do nosso leitor, otimizar seu aprendizado e instigar sua curiosidade. Os exemplos abaixo foram construídos a partir de diferentes *datasets* do repositório [vega_datasets](https://github.com/vega/vega-datasets).

### O *Dataset* Iris

Para esse exemplo usaremos o *dataset* `iris` do repositório vega_datasets. Começaremos olhando a descrição desse *dataset*:

In [73]:
data.iris.description

'This classic dataset contains lengths and widths of petals and sepals for 150 iris flowers, drawn from three species. It was introduced by R.A. Fisher in 1936 [1]_.'

"Esse conjunto de dados clássico contém comprimentos e larguras de pétalas e sépalas de 150 flores de íris, provenientes de três espécies. Ele foi introduzido por R.A. Fisher em 1936 [1]_."

Essa descrição nos permite ter uma noção melhor de quais variáveis temos disponíveis, quais os tipos de dados que podemos trabalhar, o tamanho do *data frame*, etc. Vamos então importar esse *dataset* para comerçarmos a análise:

In [74]:
iris = data.iris()
iris.head()

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


Assim, como mostrado no notebook anterior, podemos renomear as colunas para melhor entendimento, usando `rename(columns={})`.

In [75]:
iris = iris.rename(columns={
    'sepalLength': 'Comprimento da Sépala',
    'sepalWidth': 'Largura da Sépala',
    'petalLength': 'Comprimento da Pétala',
    'petalWidth': 'Largura da Pétala',
    'species': 'Espécie'
})
iris.head()

,Comprimento da Sépala,Largura da Sépala,Comprimento da Pétala,Largura da Pétala,Espécie
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


Sobre os tipos de dados, temos dados quantitivos de largura e comprimento, tanto da pétala quando da sépala, e dados nominais da espécie.

Vamos usar o atributo `describe()` de um *data frame* Pandas para obter algumas medidas de resumo, como média, mediana e quartis, para as variáveis quantitativas.

In [76]:
iris.describe()

,Comprimento da Sépala,Largura da Sépala,Comprimento da Pétala,Largura da Pétala
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


Podemos também definir um esquema de cores específico para os gráficos, para combinar melhor com o tema e contexto. Lembrando que as cores para os gráficos que optamos por fazer abaixo, não devem formar gradiente ou expressar noção de hierarquia, uma vez que essa relação não existe, e não é explorada, entre as espécies trabalhadas. Para saber melhor como é feito, consulte o notebook anterior.

In [77]:
color_map = alt.Scale(
    domain = ['setosa', 'versicolor', 'virginica'],
    range = ['#8FBED6', '#93C572', '#8A2BE2']
)

Vamos começar fazendo gráficos de dispersão relacionando as variáveis de largura e comprimento para a sépala e para a pétala.

In [78]:
disp1 = alt.Chart(iris).mark_point(filled=True).encode(
    alt.X('Comprimento da Pétala'),
    alt.Y('Largura da Pétala'),
    alt.Color('Espécie:N', scale=color_map)
).properties(
    width=200,
    height=200
)

disp2 = alt.Chart(iris).mark_point(filled=True).encode(
    alt.X('Comprimento da Sépala'),
    alt.Y('Largura da Sépala'),
    alt.Color('Espécie:N', scale=color_map)
).properties(
    width=200,
    height=200
)

alt.hconcat(disp1, disp2).properties(title={
    'text': 'Gráficos de dispersão para as características da Pétala e da Sépala',
    'anchor': 'middle'
})

alt.HConcatChart(...)

Podemos observar no gráfico de dispersão para a Pétala que as espécies *versicolor* e *virginica* se encontram próxima e, em partes sobrepostas, na distrubuição. Já a espécie *setosa*, se comporta como um cluster a parte. 

Podemos examinar melhor a distribuição da largura e comprimento da pétala entre as espécies usando um gráfico *jitter*.

Um gráfico de jitter é uma variante do gráfico de faixas (*scatter plot* com *binning*) com uma visão melhor dos pontos de dados sobrepostos, usado para visualizar a distribuição de muitos valores unidimensionais individuais. Os valores são plotados como pontos ao longo de um eixo e, em seguida, os pontos são deslocados aleatoriamente ao longo do outro eixo, o que, por si só, não tem significado em termos de dados, permitindo que os pontos não se sobreponham.

In [79]:
comp_jitter = alt.Chart(iris, title='Jitter da distribuição do comprimento da pétala').mark_circle(size=8).encode(
    y="Espécie:N",
    x="Comprimento da Pétala:Q",
    yOffset="jitter:Q",
    color=alt.Color('Espécie:N', scale=color_map).legend(None)
).transform_calculate(
    jitter="random()"
)

larg_jitter = alt.Chart(iris, title='Jitter da distribuição da largura da pétala').mark_circle(size=8).encode(
    y="Espécie:N",
    x="Largura da Pétala:Q",
    yOffset="jitter:Q",
    color=alt.Color('Espécie:N', scale=color_map).legend(None)
).transform_calculate(
    jitter="random()"
)

(comp_jitter | larg_jitter)

alt.HConcatChart(...)

Estatisticamente, podemos usar uma análise bidemensional para relacionar as duas variáveis quantitativas de Largura e Comprimento. Fazemos isso por meio de uma tabela de pivôs, usando diferentes funções de agregação, abaixo contamos as ocorrências de cada tamanho para cada espécie usando `size`.

Usamos o método `pivot_table()` do Pandas, como abaixo:

In [80]:
# Filtra as espécies versicolor e virginica
iris_filtered = iris[iris['Espécie'].isin(['versicolor', 'virginica'])]

# Tabela de pivôs para o comprimento da pétala
petal_length_pivot = iris_filtered.pivot_table(index='Comprimento da Pétala', columns='Espécie', aggfunc='size', fill_value=0)
petal_length_pivot = petal_length_pivot.reset_index()  # Torna 'Comprimento da Pétala' uma coluna regular para o Altair

# Tabela de pivôs para a largura da pétala
petal_width_pivot = iris_filtered.pivot_table(index='Largura da Pétala', columns='Espécie', aggfunc='size', fill_value=0)
petal_width_pivot = petal_width_pivot.reset_index() # Torna 'Largura da Pétala' uma coluna regular para o Altair

In [81]:
petal_length_pivot.head()

Espécie,Comprimento da Pétala,versicolor,virginica
0,3.0,1,0
1,3.3,2,0
2,3.5,2,0
3,3.6,1,0
4,3.7,1,0


Podemos visualizar as frequências com um histograma ou gráfico de barras:

In [82]:
alt.Chart(iris_filtered).mark_bar().encode(
    alt.X('Comprimento da Pétala'),
    alt.Y('count()'),
    alt.Color('Espécie', scale=color_map),
    tooltip='Comprimento da Pétala'
)

alt.Chart(...)

Fazemos o mesmo para a largura:

In [83]:
petal_width_pivot.head()

Espécie,Largura da Pétala,versicolor,virginica
0,1.0,7,0
1,1.1,3,0
2,1.2,5,0
3,1.3,13,0
4,1.4,7,1


Podemos visualizar as frequências com um histograma ou gráfico de barras:

In [84]:
alt.Chart(iris_filtered).mark_bar().encode(
    alt.X('Largura da Pétala', bin=alt.BinParams(maxbins=15)),
    alt.Y('count()'),
    alt.Color('Espécie', scale=color_map),
    tooltip='Largura da Pétala'
)

alt.Chart(...)

Além disso, podemos usar diferentes função de agregação para obter diferentes comparações entre os dados. Alguns exemplos são:

*   `'mean'` (padrão): Calcula a média (valor médio) dos valores.
*   `'sum'`: Calcula a soma dos valores.
*   `'count'`: Conta o número de valores não ausentes.
*   `'min'`: Encontra o valor mínimo.
*   `'max'`: Encontra o valor máximo.
*   `'median'`: Calcula a mediana (valor do meio) dos valores.
*   `'std'`: Calcula o desvio padrão.
*   `'var'`: Calcula a variância.
*   `'prod'`: Calcula o produto de todos os valores.
*   `'first'`: Retorna o primeiro valor.
*   `'last'`: Retorna o último valor.


Abaixo vemos as comparações a respeito da média de cada parâmetro para cada espécie.

In [85]:
pd.pivot_table(iris_filtered, values=['Comprimento da Pétala', 'Largura da Pétala'], index=['Espécie'], aggfunc='mean')

,Comprimento da Pétala,Largura da Pétala
Espécie,,
versicolor,4.260,1.326
virginica,5.552,2.026


Uma análise mais detalhada é feita usando a medida dos 5 números:

In [86]:
iris_filtered[['Comprimento da Pétala', 'Largura da Pétala']].describe()

,Comprimento da Pétala,Largura da Pétala
count,100.000000,100.000000
mean,4.906000,1.676000
std,0.825578,0.424769
min,3.000000,1.000000
25%,4.375000,1.300000
50%,4.900000,1.600000
75%,5.525000,2.000000
max,6.900000,2.500000


Podemos visualizar visualmente com um *boxplot*:

In [87]:
boxplot_comp = alt.Chart(iris_filtered).mark_boxplot().encode(
    alt.X('Espécie'),
    alt.Y('Comprimento da Pétala'),
    alt.Color('Espécie', scale=color_map)
).properties(
    width = 200,
    height = 250,
)

boxplot_larg = alt.Chart(iris_filtered).mark_boxplot().encode(
    alt.X('Espécie'),
    alt.Y('Largura da Pétala'),
    alt.Color('Espécie', scale=color_map)
).properties(
    width = 200,
    height = 250,
)

(boxplot_comp | boxplot_larg).properties(title={
    'text': 'Boxplots do comprimento e da largura da Pétala para as espécies versicolor e virginica',
    'anchor': 'middle'
})

alt.HConcatChart(...)

Outras análise a respeito das diferentes distribuições e relações entre as variáveis disponíveis foram feitas em um notebook anterior.

A partir dessas medidas de resumo, vemos que os valores apresentam concentração em intervalos específicos para as parâmetros da pétala. A fim de obter uma visualização dessas relações, analisaremos por meio de um *heatmap*, as características da pétala para as as duas espécies.

Para isso, criaremos um outro *data frame* apenas com os dados dessas espécies para análise.

In [88]:
iris_petal = iris[['Comprimento da Pétala', 'Largura da Pétala', 'Espécie']]

In [89]:
iris_petal_versi = (iris_petal.loc[iris_petal['Espécie']=='versicolor']).rename(columns={
    'Comprimento da Pétala': 'Comprimento Versicolor',
    'Largura da Pétala': 'Largura Versicolor',
    'Espécie': 'Espécie'
}).reset_index(drop=True).drop('Espécie', axis=1)

iris_petal_virg = (iris_petal.loc[iris_petal['Espécie']=='virginica']).rename(columns={
    'Comprimento da Pétala': 'Comprimento Virginica',
    'Largura da Pétala': 'Largura Virginica',
    'Espécie': 'Espécie'
}).reset_index(drop=True).drop('Espécie', axis=1)

In [90]:
iris_petal_versi_virg = pd.concat([iris_petal_versi, iris_petal_virg], axis=1)
iris_petal_versi_virg.head()

,Comprimento Versicolor,Largura Versicolor,Comprimento Virginica,Largura Virginica
0,4.7,1.4,6.0,2.5
1,4.5,1.5,5.1,1.9
2,4.9,1.5,5.9,2.1
3,4.0,1.3,5.6,1.8
4,4.6,1.5,5.8,2.2


In [91]:
heatmap_comp = alt.Chart(iris_petal_versi_virg, title='Comprimento da Pétala').mark_bar().encode(
    alt.X('Comprimento Versicolor:Q', bin=alt.BinParams(maxbins=10)),
    alt.Y('Comprimento Virginica:Q', bin=alt.BinParams(maxbins=10)),
    alt.Color('count()', title='Número de registros', scale=alt.Scale(scheme='bluepurple'))
).properties(
    width=200,
    height=200
)

heatmap_larg = alt.Chart(iris_petal_versi_virg, title='Largura da Pétala').mark_bar().encode(
    alt.X('Largura Versicolor:Q', bin=alt.BinParams(maxbins=10)),
    alt.Y('Largura Virginica:Q', bin=alt.BinParams(maxbins=10)),
    alt.Color('count()', title='Número de registros', scale=alt.Scale(scheme='bluepurple'))
).properties(
    width=200,
    height=200
)

alt.hconcat(heatmap_comp, heatmap_larg).properties(
    title={'text':'Mapas de calor das caracterísiticas da Pétala', 
           'anchor': 'middle', 
           'fontSize': 20}
    )

alt.HConcatChart(...)

Confirmando nossas hipóteses, os valores apresentam alguma concentração, representada visualmente pelas caixas em tonalidade roxo escuro.